## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_43.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_43.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩

In [4]:
Total_categorical_columns = [
    "시술_당시_나이",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, 
                                                                            Total_y, 
                                                                            test_size=0.2, 
                                                                            random_state=42,
                                                                            stratify=Total_y)

### Total 데이터

optuna

In [7]:
from catboost import CatBoostClassifier

In [8]:
# 목적 함수 정의
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.0005, 0.5),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.01, 100, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100),
        'border_count': trial.suggest_int('border_count', 32, 256),

        'thread_count': -1,  # 병렬 처리에 사용할 스레드 수 (-1이면 모든 코어 사용)
        'random_state': 42,
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }

    model = CatBoostClassifier(**param)
    model.fit(Total_X_train, Total_y_train)

    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]

    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-19 10:16:39,008] A new study created in memory with name: no-name-030b4451-9186-4850-a9d9-0f0da963e5ef
[I 2025-02-19 10:17:11,301] Trial 0 finished with value: 0.7305452539680686 and parameters: {'iterations': 1295, 'learning_rate': 0.2877816335540664, 'min_data_in_leaf': 16, 'l2_leaf_reg': 54.90542902913716, 'random_strength': 6.335173944665097, 'bagging_temperature': 67.03407485328907, 'border_count': 110}. Best is trial 0 with value: 0.7305452539680686.
[I 2025-02-19 10:17:49,052] Trial 1 finished with value: 0.7242384665042769 and parameters: {'iterations': 1234, 'learning_rate': 0.2847689956615349, 'min_data_in_leaf': 18, 'l2_leaf_reg': 1.6552477599702686, 'random_strength': 4.837575221226689, 'bagging_temperature': 4.987440969592956, 'border_count': 256}. Best is trial 0 with value: 0.7305452539680686.
[I 2025-02-19 10:18:50,181] Trial 2 finished with value: 0.7176065461268745 and parameters: {'iterations': 2075, 'learning_rate': 0.45547793305666895, 'min_data_in_leaf'

KeyboardInterrupt: 

.